### Popcorn 데이터 세트

* 데이터 전처리
* 임베딩(Word2Vec)
* 모델 : Logistic Regression

In [1]:
import os
import logging

import pandas as pd
import numpy as np

from gensim.models import word2vec # word2vec 사용
from sklearn.model_selection import train_test_split # 데이터 셋 분할
from sklearn.linear_model import LogisticRegression # 모델 회귀 모형 이용

### [1] 데이터 세트 불러오기

In [2]:
BASIC_PATH = "/dataset"

In [3]:
DATA_CLEAN_DATA = BASIC_PATH + "/train_clean.csv"

In [4]:
train_data = pd.read_csv(DATA_CLEAN_DATA)

In [5]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

* word2vec을 사용하기 위해 각 문서에서 단어들을 모두 띄어쓰기 기준으로 배열로 저장한다

In [7]:
sentences = []
for review in reviews:
    sentences.append(review.split())

In [8]:
print(sentences[0])

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

### [2] 임베딩(word2vec)

In [9]:
# 학습 시 필요한 하이퍼 파라미터

num_features = 300 # 워드 백터 특정값 수 -> 임베딩 차원 수 지정
min_word_count = 40 # 단어에 대한 최소 빈도 수 -> 빈도수가 적은 단어들은 학습 x
num_workers = 4 # 프로세스 개수 
context = 10 # 컨텍스트 윈도우 크기
downsampling = 1e-3 # 다운 샘플링 비율

* 로그 수준을 INFO로 하면 word2vec의 학습과정에서 로그 메시지를 양식에 맞게 INFO 수준으로 볼 수 있다.

In [10]:
logging.basicConfig(format='%(astime)s : %(levelname)s : %(messages)s', level=logging.INFO)

In [11]:
model = word2vec.Word2Vec(sentences, workers=num_workers, \
                         size=num_features, min_count = min_word_count, \
                         window = context, sample = downsampling)

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 622, in format
    s = self.formatMessage(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 591, in formatMessage
    return self._style.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 433, in format
    return self._fmt % record.__dict__
KeyError: 'astime'
Call stack:
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.

In [14]:
print("vocab length : %d"%len(model.wv.vocab))

vocab length : 8160


In [16]:
model.wv.most_similar('mother')

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 1034, in emit
    msg = self.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 880, in format
    return fmt.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 622, in format
    s = self.formatMessage(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 591, in formatMessage
    return self._style.format(record)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\logging\__init__.py", line 433, in format
    return self._fmt % record.__dict__
KeyError: 'astime'
Call stack:
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\msi\appdata\local\programs\python\python37\lib\runpy.

[('son', 0.8962161540985107),
 ('father', 0.8961231112480164),
 ('daughter', 0.8863471150398254),
 ('sister', 0.848844051361084),
 ('grandmother', 0.8162511587142944),
 ('husband', 0.8097283840179443),
 ('marriage', 0.8073601126670837),
 ('wife', 0.7999267578125),
 ('paulie', 0.7947574853897095),
 ('child', 0.7851319313049316)]

* word2vec 결과 : 'training on a %i raw words (%i effective words) took %.1fs, %.0f effective words/s' Arguments: (14940445, 12472454, 14.632472099999973, 852381.8746936154)

* get_feature()함수 : 

        - 지금 현재 word2vec모델을 통해 각 단어가 벡터로 표현 되어 있다(즉, 어휘 사전-코퍼스 형태)
        - 현재 리뷰마다 단어의 개수(학습할 데이터 길이)가 다르기 때문에 이를 통일해줘야 한다
        - 가장 단순한 방법은 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 것이다
        - 이 함수가 전체 단어의 평균 값을 계산하는 것이다

In [21]:
print(len(sentences[0]))
temp = get_feature(sentences[0], model, num_features)
print(temp.shape)

219
(300,)


In [22]:
print(len(sentences[1]))
temp = get_feature(sentences[1], model, num_features)
print(temp.shape)

84
(300,)


In [18]:
def get_feature(words, model, num_feature):
    # 우리는 num_feature 차원의 벡터 크기를 만들거임
    feature_vector = np.zeros((num_features), dtype=np.float32) # feature_vector 초기화
    
    num_words = 0
    
    # index2word_set -> 문장 속 단어가 모델 단어 사전에 있는지 보기 위해 어휘 사전 부름
    index2word_set = set(model.wv.index2word) # 어휘 사전 준비
    
    for w in words:
        if w in index2word_set: # 단어가 모델 단어 사전에 있으면
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[w]) #wv[w] -> 해당 단어의 임베딩 벡터 가져와 모든 벡터들의 합을 구함
            
    feature_vector = np.divide(feature_vector, num_words) # 평균을 내기 위해서 feature vector / num_words(단어 사전에 있는 단어들)한다
    return feature_vector

In [23]:
def get_dataset(sentences, model, num_features):
    dataset = list()
    
    for s in sentences: # 한 문서씩 꺼냄 (단어 배열 형태 임)
        dataset.append(get_feature(s, model, num_features)) # 이 단어 배열들을 임베딩 함
        
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

In [24]:
# train_data set 벡터화-임베딩ㅇ 진행 -> 실제 학습에 사용될 입력값을 만들어 보자.
train_data_vecs = get_dataset(sentences, model, num_features)

In [33]:
print(train_data_vecs[0])
print(len(train_data_vecs[0]))

[-4.50477898e-02 -1.64771989e-01 -5.36102839e-02 -6.96490631e-02
  5.74803464e-02 -1.91026524e-01 -4.77679744e-02  2.00568452e-01
 -1.45472303e-01  1.12178460e-01  5.92331886e-02 -9.95574668e-02
 -1.05038345e-01  1.16406478e-01 -7.74854422e-02  4.72529307e-02
 -1.48271099e-01 -8.05422105e-03 -2.00033203e-01 -4.64572012e-02
 -4.88153510e-02 -3.27002347e-01 -1.68903574e-01 -1.04110911e-01
 -1.81324199e-01 -3.80740643e-01 -1.24149181e-01  1.66915581e-01
 -1.22336067e-01  3.34876142e-02 -1.07462786e-01  1.29544020e-01
  1.20669946e-01  1.75049558e-01 -7.17401728e-02  1.09149583e-01
 -1.81916997e-01 -7.15553239e-02  1.84944749e-01 -1.74983934e-01
  1.27342567e-01  5.37931360e-02 -7.31566176e-02  6.93394914e-02
  1.58845752e-01  2.42596254e-01 -1.84531525e-01  4.02807504e-01
 -2.31821667e-02  5.68031073e-02  1.50186743e-03 -7.96071440e-03
 -9.10125896e-02 -2.54120678e-02 -3.34085375e-02 -1.87649578e-01
 -3.11757419e-02 -8.23884457e-03 -9.66083035e-02 -1.98024333e-01
  6.71970174e-02  7.63636

In [34]:
print(train_data_vecs.shape) # (25000, 300)

(25000, 300)


### [3] 데이터 세트 분할 (Train + Validation)

In [25]:
X = train_data_vecs
y = np.array(sentiments)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                  random_state=RANDOM_SEED)

### [4] 모델 선언 및 훈련

In [26]:
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

c:\users\msi\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

### [5] 모델 예측(정확도 측정)

In [27]:
predicted = lgs.predict(X_val)

In [28]:
print("Accuracy : %f" % lgs.score(X_val, y_val)) # score()함수 => 정확도 측정

Accuracy : 0.865800


### [6] TEST 데이터 세트로 모델 확인(예측)

In [29]:
TEST_CLEAN_DATA = BASIC_PATH + "/test_clean.csv"
test_data = pd.read_csv(TEST_CLEAN_DATA)

In [30]:
test_data.head(5)

,review
0,naturally film main themes mortality nostalgia...
1,movie disaster within disaster film full great...
2,movie kids saw tonight child loved one point k...
3,afraid dark left impression several different ...
4,accurate depiction small time mob life filmed ...


In [60]:
test_review = list(test_data['review'])

test_sentences = list()

for review in test_review:
    test_sentences.append(review.split())

In [61]:
print(len(test_sentences))

25000


In [62]:
testDataVecs = get_dataset(test_sentences, model, num_features)

In [63]:
test_predicted = lgs.predict(testDataVecs)

In [64]:
print(test_predicted)

[1 0 1 ... 0 1 1]
